In [1]:
import numpy as np
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import re
import json
from sklearn.linear_model import Ridge
import xgboost as xgb
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import HeatMap
from sklearn.neighbors import KNeighborsRegressor

In [ ]:

df_scraping = pd.read_csv("data_scraping.csv")
df_scraping.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10050 entries, 0 to 10049
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Link            10050 non-null  object
 1   Diện tich       9032 non-null   object
 2   Phòng WC        5815 non-null   object
 3   Phòng ngủ       6125 non-null   object
 4   Giá             10050 non-null  object
 5   Hướng nhà       1021 non-null   object
 6   Hướng ban công  513 non-null    object
 7   Mô tả           10050 non-null  object
 8   Giờ đăng        10050 non-null  object
 9   Ngày đăng       10050 non-null  object
 10  Quận            10050 non-null  object
 11  Loại nhà        10050 non-null  object
dtypes: object(12)
memory usage: 942.3+ KB


In [50]:
# Dùng Anyscale để trích xuất các Mô tả, trả về các thông tin: Diện tích, PN, WC, Floors, Sổ Hồng/ Đỏ/ Pháp Lý, Giá

def get_clean_data(description):
    s = requests.Session()

    api_base = "https://api.endpoints.anyscale.com/v1"
    # Thay token với token hợp lệ của bạn
    token = "esecret_ry2ylg5ybygg7bdlfz6ii3pnrw"
    url = f"{api_base}/chat/completions"

    # Tạo một yêu cầu để gửi đến API
    body = {
      "model": "meta-llama/Meta-Llama-3-8B-Instruct",
      "messages": [
        {
          "role": "system",
          "content": "Giờ hãy trả về đúng cho tôi các giá trị sau:\ntheo định dạng json.Only the json\n\nDiện tích : x m2\nSố phòng ngủ : None/1/2/3/4/5\nSố wc : None/1/2/3/4/5\nSố tầng : 1/2/3/4/5 ( nếu None trả về 1)\nSổ hồng/sổ đỏ/pháp lý : Có/Không\nGiá tiền : x tỷ\n\noutput sẽ ra như sau :\n{\n  \"Diện tích\": \"150m2\",\n  \"Số phòng ngủ\": \"2\",\n  \"Số wc\": \"1\",\n  \"Số tầng\": \"1\",\n  \"Sổ hồng/sổ đỏ/pháp lý\": \"Có\",\n  \"Giá tiền\": \"22 tỷ\"\n}"
        },
        {
          "role": "user",
          "content": description
        }
      ],
      "temperature": 1,
      "max_tokens": 256,
      "top_p": 1,
      "frequency_penalty": 0
    }

    # Gửi yêu cầu API và trích xuất thông điệp từ phản hồi JSON
    with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
        response_data = resp.json()
        # Trích xuất thông điệp từ phản hồi JSON
        messages = response_data.get('choices', [])
        for message in messages:
            # Kiểm tra xem có tin nhắn từ trợ lý không
            if message['message']['role'] == 'assistant':
                assistant_content = message['message']['content']
                return assistant_content


In [ ]:
# Thêm thuộc tính Des (Mô tả đã được trích xuất bằng Anyscale) vào DataFrame và lưu thành file CSV để sử dụng ở bước sau
df_scraping['des'] = df_scraping['Mô tả '].apply(get_clean_data)
df_scraping.to_csv('Data_Anyscale.csv', index=False)

In [ ]:
# Mở lại File CSV
df_anyscale = pd.read_csv("Data_Anyscale.csv")
df_anyscale

,Link,Diện tich,Phòng WC,Phòng ngủ,Giá,Hướng nhà,Hướng ban công,Mô tả,Giờ đăng,Ngày đăng,Quận,Loại nhà,des
0,https://batdongsan.vn/chua-toi-30trm2-hang-ngo...,150 m2,1 WC,2 PN,3899000000 tỷ,NaN,NaN,Mô tả: \n\r\n+ Thông số cực đẹp 5 x 30m (15...,17:11,10/12/2023,Nhà Bè,Bán Nhà riêng,"{\n ""Diện tích"": ""150m2"",\n ""Số phòng ngủ"": ..."
1,https://batdongsan.vn/ban-nha-hxh-au-co-phuong...,51 m2,NaN,NaN,5.5 tỷ,NaN,NaN,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",18:40,10/12/2023,Tân Bình,Bán Nhà riêng,"{\n ""Diện tích"": ""51m2"",\n ""Số phòng ngủ"": ""..."
2,https://batdongsan.vn/sat-mat-tien-phan-dang-l...,45 m2,2 WC,2 PN,4.6 tỷ,NaN,NaN,"- Kết cấu: 1 trệt 1 lầu BTCT; 2 PN, 2 WC. Phòn...",18:56,10/12/2023,Phú Nhuận,Bán Nhà,"{\n ""Diện tích"": ""x m2"",\n ""Số phòng ngủ"": ""..."
3,https://batdongsan.vn/chu-gap-ban-truoc-tet-le...,41 m2,NaN,NaN,7.35 tỷ,NaN,NaN,"Mô tả:\n\r\n- Diện tích công nhận 41m, thực tế...",20:49,10/12/2023,Quận 5,Bán Nhà riêng,"{\n ""Diện tích"": ""66m2"",\n ""Số phòng ngủ"": ""..."
4,https://batdongsan.vn/luy-ban-bichtan-phu-dien...,96 m2,1 WC,2 PN,Thỏa thuận,NaN,NaN,- Diện tích 96m2 ngang 4.3m dài 22.5m .\n\r\n-...,14:13,07/12/2023,Tân Phú,Bán Nhà riêng,"{\n ""Diện tích"": ""96m2"",\n ""Số phòng ngủ"": n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,https://batdongsan.vn/nha-3-tang-xe-hoi-do-cua...,60 m2,3 WC,3 PN,5.4 tỷ,NaN,NaN,"Nhà thiết kế 1 trệt 1 lửng 1 lầu, gồm 3 phòng ...",14:53,25/10/2022,Tân Phú,Bán Nhà riêng,"Here is the output in JSON format:\n\n{\n ""Di..."
10046,https://batdongsan.vn/ban-nha-moi-dep-lung-lin...,60 m2,NaN,NaN,7.93 tỷ,NaN,NaN,"+ Diện tích: 4 x 15m2, vuông A4. Khu phân lô c...",14:28,25/10/2022,Gò Vấp,Bán Nhà riêng,Here is the output in JSON format:\n```\n{\n ...
10047,https://batdongsan.vn/ban-nha-moi-dep-hxh-quay...,100 m2,NaN,NaN,6 tỷ,NaN,NaN,NHÀ THIẾT KẾ ĐẸP - TẶNG FULL NỘI THẤT- HẺM XE...,13:03,25/10/2022,Thủ Đức,Bán Nhà riêng,"Here is the output in JSON format:\n\n{\n ""Di..."
10048,https://batdongsan.vn/chinh-chu-ban-nha-1t1l-d...,52 m2,2 WC,3 PN,3.48 tỷ,Bắc,Bắc,"- Nhà 1 trệt 1 lầu đúc kiên cố, 3PN, 2WC, sân ...",17:05,22/10/2022,Quận 12,Bán Nhà riêng,Here is the output in JSON format:\n```\n{\n ...


In [9]:
# Chuyển đổi thuộc tính 'des' (Mô tả) về dạng chuỗi
df_anyscale['des'].astype(str)

0        {\n  "Diện tích": "150m2",\n  "Số phòng ngủ": ...
1        {\n  "Diện tích": "51m2",\n  "Số phòng ngủ": "...
2        {\n  "Diện tích": "x m2",\n  "Số phòng ngủ": "...
3        {\n  "Diện tích": "66m2",\n  "Số phòng ngủ": "...
4        {\n  "Diện tích": "96m2",\n  "Số phòng ngủ": n...
                               ...                        
10045    Here is the output in JSON format:\n\n{\n  "Di...
10046    Here is the output in JSON format:\n```\n{\n  ...
10047    Here is the output in JSON format:\n\n{\n  "Di...
10048    Here is the output in JSON format:\n```\n{\n  ...
10049    Here is the output in JSON format:\n\n{\n  "Di...
Name: des, Length: 10050, dtype: object

In [11]:
# Do ta sẽ sử dụng ký tự "}" như một ký tự để kết thúc chuỗi truy xuất nhưng các Des(Mô tả) lại không hoàn toàn có ký tự này
# nên ta sẽ cần hàm để kiểm tra và thêm ký tự này vào chuỗi Des (Mô tả).

def check_and_add_closing_brace(json_string):
    if not isinstance(json_string, str):
        raise ValueError("Input must be a string.")

    stripped_string = json_string.strip()
    if not stripped_string.endswith('}'):
        return stripped_string + '\n}'

    return stripped_string

df_anyscale['des'] = df_anyscale['des'].apply(check_and_add_closing_brace)

In [13]:
# Trích xuất các thông tin từ des (mô tả)

def extract_json(input_string):
    """
    Trích xuất và chuyển đổi phần JSON từ chuỗi đầu vào thành một đối tượng Python hợp lệ.

    Args:
        input_string (str): Chuỗi đầu vào chứa phần JSON.

    Returns:
        dict: Đối tượng Python đại diện cho phần JSON đã trích xuất.
    """
    # Biểu thức chính quy để tìm phần JSON
    pattern = r'\{[^}]*\}'

    # Tìm kiếm phần JSON trong chuỗi
    match = re.search(pattern, input_string, re.DOTALL)

    if match:
        # Lấy phần JSON từ kết quả tìm kiếm
        json_data = match.group()

        # Đảm bảo rằng các dấu ngoặc kép kép được thay thế bằng dấu ngoặc kép đơn
        json_data = json_data.replace('""', '"')

        # Thay thế 'None' bằng 'null' để JSON hợp lệ
        json_data = json_data.replace('None', 'null')

        try:
            # Chuyển đổi JSON thành đối tượng Python
            data = json.loads(json_data)
            return data
        except json.JSONDecodeError as e:
            # Trả về một đối tượng JSON mặc định nếu có lỗi cú pháp
            return {
                "Diện tích": None,
                "Số phòng ngủ": None,
                "Số wc": None,
                "Số tầng": None,
                "Sổ hồng/sổ đỏ/pháp lý": None,
                "Giá tiền": None
            }
    else:
        # Trả về một đối tượng JSON mặc định nếu không tìm thấy phần JSON
        return {
            "Diện tích": None,
            "Số phòng ngủ": None,
            "Số wc": None,
            "Số tầng": None,
            "Sổ hồng/sổ đỏ/pháp lý": None,
            "Giá tiền": None
        }

df_anyscale['des'] = df_anyscale['des'].apply(extract_json)

In [15]:
# Tạo Colums mota từ des (Mô tả) của Dataframe
mota = df_anyscale['des']
mota

0        {'Diện tích': '150m2', 'Số phòng ngủ': '2', 'S...
1        {'Diện tích': '51m2', 'Số phòng ngủ': 'null', ...
2        {'Diện tích': 'x m2', 'Số phòng ngủ': '2', 'Số...
3        {'Diện tích': '66m2', 'Số phòng ngủ': 'null', ...
4        {'Diện tích': '96m2', 'Số phòng ngủ': None, 'S...
                               ...                        
10045    {'Diện tích': '150m2', 'Số phòng ngủ': '3', 'S...
10046    {'Diện tích': '60m2', 'Số phòng ngủ': '4', 'Số...
10047    {'Diện tích': '100m2', 'Số phòng ngủ': None, '...
10048    {'Diện tích': '112.5m2', 'Số phòng ngủ': '3', ...
10049    {'Diện tích': '40m2', 'Số phòng ngủ': '2', 'Số...
Name: des, Length: 10050, dtype: object

In [17]:
# Danh sách các chuỗi JSON
json_strings = mota.tolist()

# Kiểm tra xem tất cả các chuỗi có phải là JSON hay không
is_valid_json = all(isinstance(item, dict) for item in json_strings)

if is_valid_json:
    # Tạo DataFrame từ danh sách các từ điển
    mota_data = pd.DataFrame(json_strings)
else:
    print("Không phải tất cả các phần tử trong danh sách là chuỗi JSON hợp lệ.")

In [19]:
# Nhận xét: Do Anyscale hoạt động không theo cú pháp đã khởi tạo nên đôi lúc sẽ cho ra các tên Thuộc tính không chính xác.
# Do đó, ta cần phải xử lý vấn đề này
mota_data.columns

Index(['Diện tích', 'Số phòng ngủ', 'Số wc', 'Số tầng',
       'Sổ hồng/sổ đỏ/pháp lý', 'Giá tiền', 'Sổ hồng/sổ đỏ(pháp lý)',
       'D torpedo', 'SNumber of bedrooms', 'Number of wc', 'Number of floors',
       'BClass', 'Price', 'Sở hông/sở đỏ/pháp lý', 'Điện tích', 'DienTich',
       'SoPhongNguy', 'SoWc', 'SoTang', 'SoHoang', 'GiaTien',
       'Sổ hồng/sổ đỏ/phác lý', 'Sở hồng/sử đỏ/pháp lý', 'Giá tiên',
       'Sổ hồng/sở đỏ/pháp lý', 'Sổ hồng/sộ đỏ/pháp lý',
       'Sổ hồng/sổ đỏ/phát lý', 'Sổ hồng/sỏ đỏ/pháp lý', 'Dien Tich',
       'So Phong Nguy', 'So WC', 'So Tang', 'Sos Hoang/So Do/Phap Ly',
       'Gia Tien', 'Diềntích', 'Sổ hồng/sổ đỏ/pháo lý', 'Dien tich',
       'So phong ngu', 'So wc', 'So tang', 'Sò tầng',
       'Sổ hồng/s大全 López.RED/pháp lý', 'So Wc', 'Só Hồng/Sổ Đỏ/Pháp Lý',
       'Vị trí', 'Ưu điểm', 'Liên hệ', 'Só tầng', 'Diên tích',
       'Sô hồng/sổ đỏ/pháp lý', 'Sở phòng ngủ', 'Sở wc', 'Sở tầng',
       'Sổ hồng/s�� đỏ/pháp lý', 'So hoang phim', ' Gia tien',

In [21]:
# Tạo các cột là các thuộc tính chính: Floor, Diện tích, Giá, Pháp lý/Sổ hồng/ Sổ đỏ, PN, WC bằng cách gộp các cột có tên tương tự

tang_columns = ['Number of floors','SoTang','So Tang','So tang','Sò tầng','Só tầng','Sở tầng','SoTan','numOfFloors']
dientich_columns = ['Điện tích','DienTich','Dien Tich','Dien tich','Diên tích',' Dien Tich','Dện tích']
giatien_columns = ['Price','GiaTien','Giá tiên','Gia Tien',' Gia tien',' Gia Tien','price','_Giá tiền']
phaply_columns = ['Sổ hồng/sổ đỏ(pháp lý)','SoHoang','Sổ hồng/sổ đỏ/phác lý','Sở hồng/sử đỏ/pháp lý','Sổ hồng/sở đỏ/pháp lý', 'Sổ hồng/sộ đỏ/pháp lý','Sổ hồng/sổ đỏ/phát lý', 'Sổ hồng/sỏ đỏ/pháp lý' ,'Sổ hồng/sổ đỏ/pháo lý'
,'Sổ hồng/s大全 López.RED/pháp lý','Só Hồng/Sổ Đỏ/Pháp Lý', 'Sô hồng/sổ đỏ/pháp lý','Sổ hồng/s�� đỏ/pháp lý','So Hong/So Do/Fap Ly', 'Sở hồng/sở đỏ/pháp lý','Sổ hồng/sđỏ/pháp lý', 'Sổ Hồng/Pháp Lý','Sổ Hồng/Sổ Đỏ/Pháp Lý', 'Sổ hồng/sổ đỏ/phép lý',
'Sổ hồng/sờ đỏ/pháp lý','SoHoang/SoDo/PhapLy', 'Sổ hồng/sดร/pháp lý','Sổ Hồng/sổ đỏ/pháp lý', 'So Hong/So Do/Phap Ly','Sổ hồng/sゝ đỏ/pháp lý','Sổ hồng/sổ đỏ/phátel lý','Sor hong/sor do/phap ly', 'Sổ hồng/sổ đỏ/pháψ lý']
sophongngu_columns = ['SNumber of bedrooms','SoPhongNguy','So Phong Nguy','So phong ngu','Sở phòng ngủ','SoPhongNgu','Số phòng ngũ','Num phòng ngủ']
sowc_columns = ['Number of wc','SoWc','So wc','So WC','Sở wc','So Wc','ソWC']

# Tạo một List là tên của các cột có tên không chính xác để có thể loại bỏ
drop_columns = [ 'Sổ hồng/sổ đỏ(pháp lý)',
       'D torpedo', 'SNumber of bedrooms', 'Number of wc', 'Number of floors',
       'BClass', 'Price', 'Sở hông/sở đỏ/pháp lý', 'Điện tích', 'DienTich',
       'SoPhongNguy', 'SoWc', 'SoTang', 'SoHoang', 'GiaTien',
       'Sổ hồng/sổ đỏ/phác lý', 'Sở hồng/sử đỏ/pháp lý', 'Giá tiên',
       'Sổ hồng/sở đỏ/pháp lý', 'Sổ hồng/sộ đỏ/pháp lý',
       'Sổ hồng/sổ đỏ/phát lý', 'Sổ hồng/sỏ đỏ/pháp lý', 'Dien Tich',
       'So Phong Nguy', 'So WC', 'So Tang', 'Sos Hoang/So Do/Phap Ly',
       'Gia Tien', 'Diềntích', 'Sổ hồng/sổ đỏ/pháo lý', 'Dien tich',
       'So phong ngu', 'So wc', 'So tang', 'Sò tầng',
       'Sổ hồng/s大全 López.RED/pháp lý', 'So Wc', 'Só Hồng/Sổ Đỏ/Pháp Lý',
       'Vị trí', 'Ưu điểm', 'Liên hệ', 'Só tầng', 'Diên tích',
       'Sô hồng/sổ đỏ/pháp lý', 'Sở phòng ngủ', 'Sở wc', 'Sở tầng',
       'Sổ hồng/s�� đỏ/pháp lý', 'So hoang phim', ' Gia tien', ' Dien Tich',
       'So Hong/So Do/Fap Ly', 'Sở hồng/sở đỏ/pháp lý', 'Địa chỉ',
       'Tổng diện tích sử dụng', 'Sổ hồng/sđỏ/pháp lý', 'Sổ Hồng/Pháp Lý',
       'Sổ Hồng/Sổ Đỏ/Pháp Lý', 'Sổ hồng/sổ đỏ/phép lý',
       'Sổ hồng/sờ đỏ/pháp lý', 'Dện tích', 'SoPhongNgu', 'ソWC', 'SoTan',
       'SoHoang/SoDo/PhapLy', 'D.setTextureext', 'Sổ hồng/sดร/pháp lý',
       'Kết cấu', 'Dân trí', 'Sổ Hồng/sổ đỏ/pháp lý', 'So Hong/So Do/Phap Ly',
       ' Gia Tien', 'numberOfBedrooms', 'numberOfWC', 'numOfFloors',
       'sellinformation', 'price', 'Số phòng ngũ', 'Sổ hồng/sゝ đỏ/pháp lý',
       'Sổ hồng/sổ đỏ/phátel lý', 'Num phòng ngủ', 'Dैन tích',
       'Sor hong/sor do/phap ly', 'Sổ hồng/sổ đỏ/pháψ lý', '_Giá tiền',
       'Ngày đăng']


In [23]:
# Ta sẽ fill các giá trị null của DataFrame mota_data bằng các cột chính đã tạo ở trên theo từng index
mota_data['Số tầng'] = mota_data['Số tầng'].fillna(mota_data[tang_columns].ffill(axis=1).iloc[:, -1])
mota_data['Số phòng ngủ'] = mota_data['Số phòng ngủ'].fillna(mota_data[sophongngu_columns].ffill(axis=1).iloc[:, -1])
mota_data['Số wc'] = mota_data['Số wc'].fillna(mota_data[sowc_columns].ffill(axis=1).iloc[:, -1])
mota_data['Diện tích'] = mota_data['Diện tích'].fillna(mota_data[dientich_columns].ffill(axis=1).iloc[:, -1])
mota_data['Sổ hồng/sổ đỏ/pháp lý'] = mota_data['Sổ hồng/sổ đỏ/pháp lý'].fillna(mota_data[phaply_columns].ffill(axis=1).iloc[:, -1])
mota_data['Giá tiền'] = mota_data['Giá tiền'].fillna(mota_data[giatien_columns].ffill(axis=1).iloc[:, -1])

mota_data.drop(columns = drop_columns, inplace=True)

In [25]:
# Ta được kết quả data trích xuất từ Mô tả ở dạng Dataframe cuối cùng
mota_data

,Diện tích,Số phòng ngủ,Số wc,Số tầng,Sổ hồng/sổ đỏ/pháp lý,Giá tiền
0,150m2,2,1,1,Có,22 tỷ
1,51m2,null,null,3,Có,5 tỷ
2,x m2,2,2,2,Có,x tỷ
3,66m2,null,null,2,Có,15 tỷ/tháng
4,96m2,NaN,NaN,2,Có,NaN
...,...,...,...,...,...,...
10045,150m2,3,null,3,Có,22 tỷ
10046,60m2,4,1,5,Có,22 tỷ
10047,100m2,NaN,NaN,1,Có,6 tỷ
10048,112.5m2,3,2,2,Có,"3,480 tỷ"


In [27]:
# Kết hợp 2 dataframe theo từng dòng, ta sẽ được một Dataframe cuối cùng
raw_data = pd.merge(df_anyscale, mota_data, left_index=True, right_index=True)
raw_data

,Link,Diện tich,Phòng WC,Phòng ngủ,Giá,Hướng nhà,Hướng ban công,Mô tả,Giờ đăng,Ngày đăng,Quận,Loại nhà,des,Diện tích,Số phòng ngủ,Số wc,Số tầng,Sổ hồng/sổ đỏ/pháp lý,Giá tiền
0,https://batdongsan.vn/chua-toi-30trm2-hang-ngo...,150 m2,1 WC,2 PN,3899000000 tỷ,NaN,NaN,Mô tả: \n\r\n+ Thông số cực đẹp 5 x 30m (15...,17:11,10/12/2023,Nhà Bè,Bán Nhà riêng,"{'Diện tích': '150m2', 'Số phòng ngủ': '2', 'S...",150m2,2,1,1,Có,22 tỷ
1,https://batdongsan.vn/ban-nha-hxh-au-co-phuong...,51 m2,NaN,NaN,5.5 tỷ,NaN,NaN,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",18:40,10/12/2023,Tân Bình,Bán Nhà riêng,"{'Diện tích': '51m2', 'Số phòng ngủ': 'null', ...",51m2,null,null,3,Có,5 tỷ
2,https://batdongsan.vn/sat-mat-tien-phan-dang-l...,45 m2,2 WC,2 PN,4.6 tỷ,NaN,NaN,"- Kết cấu: 1 trệt 1 lầu BTCT; 2 PN, 2 WC. Phòn...",18:56,10/12/2023,Phú Nhuận,Bán Nhà,"{'Diện tích': 'x m2', 'Số phòng ngủ': '2', 'Số...",x m2,2,2,2,Có,x tỷ
3,https://batdongsan.vn/chu-gap-ban-truoc-tet-le...,41 m2,NaN,NaN,7.35 tỷ,NaN,NaN,"Mô tả:\n\r\n- Diện tích công nhận 41m, thực tế...",20:49,10/12/2023,Quận 5,Bán Nhà riêng,"{'Diện tích': '66m2', 'Số phòng ngủ': 'null', ...",66m2,null,null,2,Có,15 tỷ/tháng
4,https://batdongsan.vn/luy-ban-bichtan-phu-dien...,96 m2,1 WC,2 PN,Thỏa thuận,NaN,NaN,- Diện tích 96m2 ngang 4.3m dài 22.5m .\n\r\n-...,14:13,07/12/2023,Tân Phú,Bán Nhà riêng,"{'Diện tích': '96m2', 'Số phòng ngủ': None, 'S...",96m2,NaN,NaN,2,Có,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,https://batdongsan.vn/nha-3-tang-xe-hoi-do-cua...,60 m2,3 WC,3 PN,5.4 tỷ,NaN,NaN,"Nhà thiết kế 1 trệt 1 lửng 1 lầu, gồm 3 phòng ...",14:53,25/10/2022,Tân Phú,Bán Nhà riêng,"{'Diện tích': '150m2', 'Số phòng ngủ': '3', 'S...",150m2,3,null,3,Có,22 tỷ
10046,https://batdongsan.vn/ban-nha-moi-dep-lung-lin...,60 m2,NaN,NaN,7.93 tỷ,NaN,NaN,"+ Diện tích: 4 x 15m2, vuông A4. Khu phân lô c...",14:28,25/10/2022,Gò Vấp,Bán Nhà riêng,"{'Diện tích': '60m2', 'Số phòng ngủ': '4', 'Số...",60m2,4,1,5,Có,22 tỷ
10047,https://batdongsan.vn/ban-nha-moi-dep-hxh-quay...,100 m2,NaN,NaN,6 tỷ,NaN,NaN,NHÀ THIẾT KẾ ĐẸP - TẶNG FULL NỘI THẤT- HẺM XE...,13:03,25/10/2022,Thủ Đức,Bán Nhà riêng,"{'Diện tích': '100m2', 'Số phòng ngủ': None, '...",100m2,NaN,NaN,1,Có,6 tỷ
10048,https://batdongsan.vn/chinh-chu-ban-nha-1t1l-d...,52 m2,2 WC,3 PN,3.48 tỷ,Bắc,Bắc,"- Nhà 1 trệt 1 lầu đúc kiên cố, 3PN, 2WC, sân ...",17:05,22/10/2022,Quận 12,Bán Nhà riêng,"{'Diện tích': '112.5m2', 'Số phòng ngủ': '3', ...",112.5m2,3,2,2,Có,"3,480 tỷ"


In [ ]:
raw_data.to_csv('raw_data.csv', index=False)